In [1]:
midifile = 'data/chopin-fantaisie.mid'

In [2]:
import time
import copy
import subprocess
from abc import abstractmethod

from midipattern import MidiPattern
from distorter import *
from align import align_frame_to_frame, read_align, write_align

# Midi file parser
import midi

device 0 ('ALSA', 'Midi Through Port-0', 0, 1, 0)
device 1 ('ALSA', 'Midi Through Port-0', 1, 0, 0)
device 2 ('ALSA', 'TiMidity port 0', 0, 1, 0)
device 3 ('ALSA', 'TiMidity port 1', 0, 1, 0)
device 4 ('ALSA', 'TiMidity port 2', 0, 1, 0)
device 5 ('ALSA', 'TiMidity port 3', 0, 1, 0)


In [3]:
MidiPattern.MIDI_DEVICE = 2

Init Pygame and Audio
--------

Midi Pattern
--------

In [8]:
pattern = MidiPattern(midi.read_midifile(midifile))
simple = pattern.simplified(bpm=160)
simple.stamp_time('t0')
midi.write_midifile("generated/simple.mid", simple)

In [ ]:
print simple.attributes[0][-40:]

In [ ]:
pattern[0]
pattern.play(180)

In [ ]:
simple.play()

Distorter
--------

In [ ]:
distorter = VelocityNoiseDistorter(sigma=20.)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-noise.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
print dist_pattern.attributes[0][-4:]

In [ ]:
distorter = VelocityWalkDistorter(sigma=0.1)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/velocity-walk.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
distorter = ProgramDistorter()
distorter.randomize()
# for some reason GM 1- 3 makes no sound in pygame?
print distorter
dist_pattern = distorter.distort(simple)
midi.write_midifile('generated/program.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [ ]:
distorter = TempoDistorter(sigma=0, min=0.5, max=2.)
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
print 'time warp', dist_pattern.attributes[0][-4:]
midi.write_midifile('generated/tempo.mid', dist_pattern)
dist_pattern.play(bpm=180)

In [6]:
distorter = TimeNoiseDistorter()
distorter.randomize()
print distorter
dist_pattern = distorter.distort(simple)
print 'time warp', dist_pattern.attributes[0][-4:]
midi.write_midifile('generated/time.mid', dist_pattern)
dist_pattern.play(bpm=180)

TimeNoiseDistorter(sigma=0.01)
time warp

NameError: name 'dist_pattern' is not defined

Individual Note Times to Global Alignment
-------

In [5]:
stride = 1.
align = align_frame_to_frame(dist_pattern, stride)
align

NameError: name 'dist_pattern' is not defined

In [ ]:
write_align('generated/align.txt', align, stride)
align2, stride2 = read_align('generated/align.txt')
print align2 == align
print int(stride2) == int(stride), stride2, stride

Actual Generation
----

In [9]:
dist_pattern = random_distort(simple)
align = align_frame_to_frame(dist_pattern, stride=1.)
print align
dist_pattern.play()

NameError: global name 'simple' is not defined

In [ ]:
num_samples = 10
stride = 0.1
for i in xrange(num_samples):
    base_name = 'generated/sample-{}'.format(i)
    align_name = '{}.txt'.format(base_name)
    midi_name = '{}.mid'.format(base_name)
    wav_name = '{}.wav'.format(base_name)
    distorted = random_distort(base_name)
    align = align_frame_to_frame(distorted, stride)
    write_align(align_name, align, stride)
    midi.write_midifile(midi_name, distorted)
    # Convert to wav using timidity
    print wav_name
    subprocess.check_call(['timidity', '-Ow', midi_name, '-o', wav_name])
    print 'Done generating {}'.format(base_name)